In [1]:
%cd ..\..\..
import pandas as pd
import gc
import datetime

D:\Projects\CondaAnalyses\PMI


In [2]:
data_path = 'data_checkpoints/usecase_cab/'

In [3]:
# import the occupancy df to get the id_event feature
df = pd.read_csv(data_path+'occupancy_state_change.csv', index_col=0)
df.head(2)

,id,id_event
0,abboip_1211033530,abboip_1211033530
1,abboip_1211033614,abboip_1211033530


In [4]:
# append time column from id
df['time'] = pd.to_datetime(df['id'].apply(lambda row: row.split('_')[1]).astype(int), unit='s')
df.head()

,id,id_event,time
0,abboip_1211033530,abboip_1211033530,2008-05-17 14:12:10
1,abboip_1211033614,abboip_1211033530,2008-05-17 14:13:34
2,abboip_1211033674,abboip_1211033530,2008-05-17 14:14:34
3,abboip_1211033735,abboip_1211033530,2008-05-17 14:15:35
4,abboip_1211035303,abboip_1211033530,2008-05-17 14:41:43


In [5]:
# sort by time and group dataset by id_event
id_events_df = df.sort_values(['time']).groupby(by=['id_event'])

# create a duration df by filtering event duration
event_duration_df = (id_events_df['time'].max() - id_events_df['time'].min()).dt.total_seconds()
event_duration_df = event_duration_df.reset_index().rename(columns={'time': 'event_duration'})

# merge to df
df = df.merge(event_duration_df, on='id_event')

# make RAM happy
del event_duration_df, id_events_df
gc.collect()
df.drop(['id_event', 'time'], axis=1, inplace=True)
df.head()

,id,event_duration
0,abboip_1211033530,2314.0
1,abboip_1211033614,2314.0
2,abboip_1211033674,2314.0
3,abboip_1211033735,2314.0
4,abboip_1211035303,2314.0


In [6]:
df.to_csv(data_path+'event_duration.csv')
del df
gc.collect()

0